In [1]:
#!/usr/bin/env python3
"""
Complete MFSR scraper that captures ALL documents from all sectors.
Loops through all sector pages and extracts all documents regardless of type.
"""

import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd
import re
import sys
import traceback
import time

BASE = "https://www.mfsr.sk"

# All sector URLs to scrape
SECTOR_URLS = {
    "Obrana": "https://www.mfsr.sk/sk/financie/hodnota-za-peniaze/hodnotenia/obrana.html",
    "Budovy": "https://www.mfsr.sk/sk/financie/hodnota-za-peniaze/hodnotenia/budovy.html", 
    "Doprava": "https://www.mfsr.sk/sk/financie/hodnota-za-peniaze/hodnotenia/doprava.html",
    "Informatizacia": "https://www.mfsr.sk/sk/financie/hodnota-za-peniaze/hodnotenia/informatizacia.html",
    "Ostatne": "https://www.mfsr.sk/sk/financie/hodnota-za-peniaze/hodnotenia/ostatne.html"
}

# Regexes
date_re = re.compile(r'\b\d{1,2}\.\d{1,2}\.\d{4}\b')
size_re = re.compile(r'\b\d+(?:[.,]\d+)?\s*(?:kB|KB|MB|GB|B|kb|mb|gb)\b')

def extract_date_and_size(anchor):
    """Search for date and size strings around the anchor."""
    search_texts = []

    try:
        search_texts.append(anchor.get_text(" ", strip=True))
    except Exception:
        pass

    if anchor.parent:
        try:
            search_texts.append(anchor.parent.get_text(" ", strip=True))
        except Exception:
            pass

    for sib in list(anchor.previous_siblings)[:12]:
        if hasattr(sib, "get_text"):
            search_texts.append(sib.get_text(" ", strip=True))
        else:
            search_texts.append(str(sib).strip())

    for sib in list(anchor.next_siblings)[:6]:
        if hasattr(sib, "get_text"):
            search_texts.append(sib.get_text(" ", strip=True))
        else:
            search_texts.append(str(sib).strip())

    combined = " ".join([t for t in search_texts if t])
    date_match = date_re.search(combined)
    size_match = size_re.search(combined)

    return date_match.group(0) if date_match else None, size_match.group(0) if size_match else None

def is_document_link(href, link_text):
    """Check if link is a document (PDF, Word, Excel, PowerPoint, etc.)."""
    if not href:
        return False
    
    href_l = href.lower()
    link_text_l = link_text.lower()
    
    # Common document extensions
    document_extensions = [
        '.pdf', '.doc', '.docx', '.xls', '.xlsx', '.ppt', '.pptx',
        '.txt', '.rtf', '.odt', '.ods', '.odp', '.csv', '.xml',
        '.zip', '.rar', '.7z', '.tar', '.gz'
    ]
    
    # Check for extensions in href
    for ext in document_extensions:
        if ext in href_l:
            return True
    
    # Check for document keywords in link text
    document_keywords = [
        'pdf', 'document', 'doc', 'excel', 'spreadsheet', 'presentation',
        'powerpoint', 'word', 'text', 'file', 'download', 'attachment'
    ]
    
    for keyword in document_keywords:
        if keyword in link_text_l:
            return True
    
    return False

def determine_document_type(link_text, parent_text=""):
    """Determine document type from link text and parent context."""
    text_to_search = f"{link_text} {parent_text}".lower()
    
    # Standard types
    if 'hodnotenie' in text_to_search:
        return 'hodnotenie'
    elif 'analýza' in text_to_search or 'analyza' in text_to_search:
        return 'analýza'
    elif 'štúdia uskutočniteľnosti' in text_to_search:
        return 'štúdia uskutočniteľnosti'
    elif 'aktualizácia' in text_to_search:
        return 'aktualizácia'
    elif 'stanovisko' in text_to_search:
        return 'stanovisko'
    elif 'správa' in text_to_search:
        return 'správa'
    elif 'metodika' in text_to_search:
        return 'metodika'
    elif 'zmluva' in text_to_search:
        return 'zmluva'
    elif 'dohoda' in text_to_search:
        return 'dohoda'
    elif 'investičný zámer' in text_to_search:
        return 'investičný zámer'
    else:
        # If no specific type found, use the link text itself (truncated)
        return link_text[:50] + "..." if len(link_text) > 50 else link_text

def scrape_sector_page(sector_name, page_url, session):
    """Scrape a single sector page and return list of documents."""
    print(f"\nScraping {sector_name} sector: {page_url}")
    
    try:
        resp = session.get(page_url, timeout=20)
        if resp.status_code != 200:
            print(f"Non-200 status code for {sector_name}: {resp.status_code}")
            return []

        resp.encoding = resp.apparent_encoding or 'utf-8'
        soup = BeautifulSoup(resp.text, "lxml")

        rows = []
        seen = set()
        last_project_name = "UNKNOWN_PROJECT"

        # Loop through all headers and links in order
        for elem in soup.find_all(["h4", "h5", "a"]):
            if elem.name == "h4":
                # Skip h4 headers as we're using sector_name from the URL
                continue
            elif elem.name == "h5":
                last_project_name = elem.get_text(" ", strip=True)
            elif elem.name == "a" and is_document_link(elem.get("href"), elem.get_text(" ", strip=True)):
                link_text = elem.get_text(" ", strip=True)

                # Determine type - captures ALL document types
                parent_text = elem.parent.get_text(" ", strip=True) if elem.parent else ""
                dtype = determine_document_type(link_text, parent_text)

                url = urljoin(BASE, elem["href"])
                date, size = extract_date_and_size(elem)

                # Use URL as unique key to avoid duplicates
                key = (sector_name, last_project_name, url)
                if key not in seen:
                    rows.append((sector_name, last_project_name, dtype, url, date or "", size or ""))
                    seen.add(key)

        print(f"Found {len(rows)} documents in {sector_name}")
        return rows

    except Exception as e:
        print(f"Error scraping {sector_name}: {e}")
        return []

def main():
    """Main function to scrape all sectors."""
    try:
        session = requests.Session()
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                          "AppleWebKit/537.36 (KHTML, like Gecko) "
                          "Chrome/117.0.0.0 Safari/537.36",
            "Accept-Language": "en-US,en;q=0.9,sk;q=0.8"
        }
        session.headers.update(headers)

        all_rows = []
        
        # Scrape each sector
        for sector_name, page_url in SECTOR_URLS.items():
            sector_rows = scrape_sector_page(sector_name, page_url, session)
            all_rows.extend(sector_rows)
            
            # Small delay between requests to be respectful
            time.sleep(1)

        # Create DataFrame
        df = pd.DataFrame(all_rows, columns=["Sector", "Project Name", "Type", "URL", "Date", "File Size"])
        
        print(f"\n{'='*60}")
        print(f"SCRAPING COMPLETE")
        print(f"{'='*60}")
        print(f"Total documents collected: {len(df)}")
        
        # Show summary by sector
        print(f"\nDocuments by sector:")
        sector_counts = df['Sector'].value_counts()
        for sector, count in sector_counts.items():
            print(f"  {sector}: {count}")
        
        # Show unique document types found
        print(f"\nUnique document types found:")
        unique_types = df['Type'].value_counts()
        for doc_type, count in unique_types.items():
            print(f"  {doc_type}: {count}")
        
        # Show sample of results
        print(f"\nSample of results:")
        print(df.head(10).to_string(index=False))
        
        # Save to CSV
        output_file = "full_mfsr_data_complete.csv"
        df.to_csv(output_file, index=False, encoding="utf-8-sig")
        print(f"\nSaved complete data to: {output_file}")
        
        # Show some examples of the previously missed document types
        print(f"\nExamples of 'Aktualizácia' documents:")
        aktualizacia_docs = df[df['Type'] == 'aktualizácia']
        if not aktualizacia_docs.empty:
            print(aktualizacia_docs[['Sector', 'Project Name', 'Type', 'Date']].head().to_string(index=False))
        
        print(f"\nExamples of 'Stanovisko' documents:")
        stanovisko_docs = df[df['Type'] == 'stanovisko']
        if not stanovisko_docs.empty:
            print(stanovisko_docs[['Sector', 'Project Name', 'Type', 'Date']].head().to_string(index=False))

    except Exception as e:
        print("Exception occurred:", e)
        traceback.print_exc()
        sys.exit(1)

if __name__ == "__main__":
    main()

/Users/ghinwamoujaes/Desktop/World Bank/Code/slovakia_CaseStudy/wb_venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(



Scraping Obrana sector: https://www.mfsr.sk/sk/financie/hodnota-za-peniaze/hodnotenia/obrana.html
Found 28 documents in Obrana

Scraping Budovy sector: https://www.mfsr.sk/sk/financie/hodnota-za-peniaze/hodnotenia/budovy.html
Found 44 documents in Budovy

Scraping Doprava sector: https://www.mfsr.sk/sk/financie/hodnota-za-peniaze/hodnotenia/doprava.html
Found 148 documents in Doprava

Scraping Informatizacia sector: https://www.mfsr.sk/sk/financie/hodnota-za-peniaze/hodnotenia/informatizacia.html
Found 112 documents in Informatizacia

Scraping Ostatne sector: https://www.mfsr.sk/sk/financie/hodnota-za-peniaze/hodnotenia/ostatne.html
Found 22 documents in Ostatne

SCRAPING COMPLETE
Total documents collected: 354

Documents by sector:
  Doprava: 148
  Informatizacia: 112
  Budovy: 44
  Obrana: 28
  Ostatne: 22

Unique document types found:
  analýza: 206
  hodnotenie: 63
  štúdia uskutočniteľnosti: 52
  stanovisko: 9
  Dohody o hraničných priechodoch: 5
  Aktualizovaná štúdia: 3
  aktua

In [2]:
df = pd.read_csv('full_mfsr_data_complete.csv')


In [3]:
len(df)

354

In [4]:
len(df['Project Name'].unique())

266

In [5]:
import pandas as pd
import numpy as np
from datetime import datetime
import random

def create_unique_ids(df):
    """
    Create unique IDs for documents based on sector and chronological order.
    
    Step 1: Create an ID for each Sector
    Step 2: For each sector, create an ID that increases with the date order 
            and then increases randomly for the non-date documents
    Step 3: The Final ID will be SectorID.2ndID
    
    Args:
        df: DataFrame with columns ['Sector', 'Project Name', 'Type', 'URL', 'Date', 'File Size']
    
    Returns:
        DataFrame with additional 'Document_ID' column
    """
    
    # Step 1: Define sector mapping (SectorID)
    sector_mapping = {
        'Doprava': 1,
        'Informatizacia': 2, 
        'Budovy': 3,
        'Obrana': 4,
        'Ostatne': 5
    }
    
    # Convert Date column to datetime, handling NaN values
    def parse_date(date_str):
        if pd.isna(date_str) or date_str == '':
            return None
        try:
            # Handle Slovak date format DD.MM.YYYY
            return datetime.strptime(str(date_str), '%d.%m.%Y')
        except:
            return None
    
    df['Date_parsed'] = df['Date'].apply(parse_date)
    
    # Create a copy to work with
    df_with_ids = df.copy()
    df_with_ids['Document_ID'] = ''
    
    # Process each sector separately
    for sector, sector_id in sector_mapping.items():
        print(f"Processing sector: {sector} (SectorID: {sector_id})")
        
        # Filter dataframe to only this sector
        sector_df = df_with_ids[df_with_ids['Sector'] == sector].copy()
        
        if len(sector_df) == 0:
            print(f"  No documents found for {sector}")
            continue
            
        print(f"  Found {len(sector_df)} documents for {sector}")
        
        # Step 2: Sort by date (documents with dates first, then without dates)
        sector_df = sector_df.sort_values('Date_parsed', na_position='last')
        
        # Assign sequential IDs starting from 01 for documents with dates
        date_documents = sector_df[sector_df['Date_parsed'].notna()]
        no_date_documents = sector_df[sector_df['Date_parsed'].isna()]
        
        # Assign sequential IDs to documents with dates
        for i, (idx, row) in enumerate(date_documents.iterrows(), 1):
            # Use 3-digit formatting from the start to avoid float conversion issues
            doc_id = f"{sector_id}.{i:03d}"
            df_with_ids.loc[idx, 'Document_ID'] = doc_id
            print(f"    Assigned {doc_id} to document with date at index {idx}")
        
        # Assign random sequential IDs to documents without dates
        if len(no_date_documents) > 0:
            # Get the highest ID number used for this sector
            max_id = len(date_documents)
            
            # Create random order for no-date documents
            no_date_indices = no_date_documents.index.tolist()
            random.shuffle(no_date_indices)
            
            for i, idx in enumerate(no_date_indices):
                doc_num = max_id + i + 1
                # Use 3-digit formatting from the start
                doc_id = f"{sector_id}.{doc_num:03d}"
                df_with_ids.loc[idx, 'Document_ID'] = doc_id
                print(f"    Assigned {doc_id} to document without date at index {idx}")
    
    # Ensure Document_ID column is stored as string and force string type
    df_with_ids['Document_ID'] = df_with_ids['Document_ID'].astype(str)
    
    return df_with_ids.drop('Date_parsed', axis=1)

def main():
    # Load the data
    df = pd.read_csv('full_mfsr_data_complete.csv')
    
    print("="*60)
    print("CREATING UNIQUE DOCUMENT IDs")
    print("="*60)
    print(f"Total documents: {len(df)}")
    print(f"Sectors: {df['Sector'].value_counts().to_dict()}")
    print()
    
    # Create unique IDs
    df_with_ids = create_unique_ids(df)
    
    print("\n" + "="*60)
    print("RESULTS")
    print("="*60)
    
    # Save the result
    df_with_ids.to_csv("full_mfsr_data_completewith_ids.csv", index=False, encoding="utf-8-sig")
    print("Saved results to: full_mfsr_data_with_ids.csv")
    
    # Show summary by sector
    print("\nID ranges by sector:")
    for sector in ['Doprava', 'Informatizacia', 'Budovy', 'Obrana', 'Ostatne']:
        sector_data = df_with_ids[df_with_ids['Sector'] == sector]['Document_ID']
        if not sector_data.empty:
            print(f"{sector}: {sector_data.min()} to {sector_data.max()} ({len(sector_data)} documents)")
    
    # Show sample results
    print("\nSample of results:")
    sample_cols = ['Document_ID', 'Sector', 'Project Name', 'Date']
    print(df_with_ids[sample_cols].head(10).to_string(index=False))
    
    # Verify all documents have IDs
    missing_ids = len(df_with_ids[df_with_ids['Document_ID'] == ''])
    print(f"\nDocuments without IDs: {missing_ids}")
    print(f"Total documents processed: {len(df_with_ids)}")
    
    # Show examples by sector
    print("\nFirst 3 IDs for each sector:")
    for sector in ['Doprava', 'Informatizacia', 'Budovy', 'Obrana', 'Ostatne']:
        sector_data = df_with_ids[df_with_ids['Sector'] == sector]['Document_ID'].head(3)
        if not sector_data.empty:
            print(f"{sector}: {sector_data.tolist()}")

if __name__ == "__main__":
    main()


CREATING UNIQUE DOCUMENT IDs
Total documents: 354
Sectors: {'Doprava': 148, 'Informatizacia': 112, 'Budovy': 44, 'Obrana': 28, 'Ostatne': 22}

Processing sector: Doprava (SectorID: 1)
  Found 148 documents for Doprava
    Assigned 1.001 to document with date at index 219
    Assigned 1.002 to document with date at index 216
    Assigned 1.003 to document with date at index 218
    Assigned 1.004 to document with date at index 217
    Assigned 1.005 to document with date at index 215
    Assigned 1.006 to document with date at index 214
    Assigned 1.007 to document with date at index 213
    Assigned 1.008 to document with date at index 211
    Assigned 1.009 to document with date at index 210
    Assigned 1.010 to document with date at index 209
    Assigned 1.011 to document with date at index 208
    Assigned 1.012 to document with date at index 207
    Assigned 1.013 to document with date at index 206
    Assigned 1.014 to document with date at index 205
    Assigned 1.015 to docu